In [5]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

# 1. Create a function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load(wiki_json, kaggle_movies, kaggle_ratings):
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv(kaggle_movies, low_memory=False)
    ratings = pd.read_csv(kaggle_ratings, low_memory=False)

    # 3. Open the read the Wikipedia data JSON file.
    with open (wiki_json, 'r') as file:
        data = json.load(file)
    
    # 4. Read in the raw wiki movie data as a Pandas DataFrame.
    wiki_movies_df = pd.DataFrame(data)
    
    # 5. Return the three DataFrames
    return wiki_movies_df, kaggle_metadata, ratings

# 6 Create the path to your file directory and variables for the three files. 
file_dir = 'Resources/'
# Wikipedia data
wiki_file = f'{file_dir}/wikipedia-movies.json'
# Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

# 7. Set the three variables in Step 6 equal to the function created in Step 1.
wiki_file, kaggle_file, ratings_file = extract_transform_load(wiki_file, kaggle_file, ratings_file)

In [6]:
# 8. Set the DataFrames from the return statement equal to the file names in Step 6. 
wiki_movies_df = wiki_file
kaggle_metadata = kaggle_file
ratings = ratings_file

In [7]:
# 9. Check the wiki_movies_df DataFrame.
wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# 10. Check the kaggle_metadata DataFrame.
kaggle_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [9]:
# 11. Check the ratings DataFrame.
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


# FOR TESTING -- DELETE AFTER DONE

In [15]:
columnas = sorted(list(wiki_movies_df.columns))

In [16]:
columnas

['Actor control',
 'Adaptation by',
 'Alias',
 'Alma mater',
 'Also known as',
 'Animation by',
 'Arabic',
 'Area',
 'Area served',
 'Artist(s)',
 'Attraction type',
 'Audio format',
 'Author',
 'Based on',
 'Biographical data',
 'Bopomofo',
 'Born',
 'Box office',
 'Budget',
 'Camera setup',
 'Cantonese',
 'Characters',
 'Children',
 'Chinese',
 'Cinematography',
 'Closing date',
 'Color process',
 'Comics',
 'Composer(s)',
 'Coordinates',
 'Country',
 'Country of origin',
 'Cover artist',
 'Created by',
 'Date premiered',
 'Designer(s)',
 'Developed by',
 'Developer(s)',
 'Dewey Decimal',
 'Died',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Divisions',
 'Duration',
 'Edited by',
 'Editor(s)',
 'Ending theme',
 'Engine',
 'Engine(s)',
 'Executive producer(s)',
 'Family',
 'Fate',
 'Film(s)',
 'Followed by',
 'Format(s)',
 'Formerly',
 'Founded',
 'Founder',
 'Founders',
 'French',
 'Full name',
 'Gender',
 'Genre',
 'Genre(s)',
 'Genres',
 'Gwoyeu Romatzyh',
 'Ha

In [18]:
len(wiki_movies_df)

7311

In [20]:
with open ('Resources/wikipedia-movies.json', 'r') as file:
    data = json.load(file)

In [110]:
no_TV_shows = [movie for movie in data if 'No. of episodes' not in movie]

In [111]:
len(no_TV_shows)

7305

In [113]:
no_TV_shows_df = pd.DataFrame(no_TV_shows)
no_TV_shows_df.head(2)

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
def clean_movie (movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    # combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
        if len(alt_titles) > 0:
            movie['alt_titles'] = alt_titles
    
    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie

In [106]:
cleaned_movies = [clean_movie(movie) for movie in no_TV_shows]

In [107]:
len(cleaned_movies)

7305

In [118]:
cleaned_movies_df = pd.DataFrame(cleaned_movies)
cleaned_movies_df.head(2)

,url,year,imdb_link,title,Based on,Starring,Narrated by,Cinematography,Release date,Running time,...,Number of employees,Divisions,Subsidiaries,Predecessor,Founders,Area served,Products,Services,Revenue,Operating income
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...","Andrew ""Dice"" Clay",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",NaN,Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
sorted(list(cleaned_movies_df.columns))

['Actor control',
 'Alias',
 'Alma mater',
 'Animation by',
 'Area',
 'Area served',
 'Artist(s)',
 'Attraction type',
 'Audio format',
 'Author',
 'Based on',
 'Biographical data',
 'Bopomofo',
 'Born',
 'Box office',
 'Budget',
 'Characters',
 'Children',
 'Cinematography',
 'Closing date',
 'Color process',
 'Comics',
 'Composer(s)',
 'Coordinates',
 'Country',
 'Cover artist',
 'Created by',
 'Date premiered',
 'Designer(s)',
 'Developer(s)',
 'Dewey Decimal',
 'Died',
 'Director',
 'Distributor',
 'Divisions',
 'Duration',
 'Editor(s)',
 'Engine',
 'Engine(s)',
 'Executive producer(s)',
 'Family',
 'Fate',
 'Film(s)',
 'Followed by',
 'Format(s)',
 'Formerly',
 'Founded',
 'Founder',
 'Founders',
 'Full name',
 'Gender',
 'Genre',
 'Genre(s)',
 'Genres',
 'Gwoyeu Romatzyh',
 'Hanyu Pinyin',
 'Headquarters',
 'Height',
 'Hokkien POJ',
 'IPA',
 'ISBN',
 'Illustrator',
 'Industry',
 'Instruments',
 'Jyutping',
 'Key people',
 'LC Class',
 'Label',
 'Labels',
 'Language',
 'Literal me

In [125]:
cleaned_movies_df['imdb_link'].sample(30)

7266    https://www.imdb.com/title/tt2368254/
1068    https://www.imdb.com/title/tt0110538/
985     https://www.imdb.com/title/tt0109783/
778     https://www.imdb.com/title/tt0107211/
673     https://www.imdb.com/title/tt0105665/
2254    https://www.imdb.com/title/tt0119988/
76      https://www.imdb.com/title/tt0099581/
6870    https://www.imdb.com/title/tt2568862/
1195    https://www.imdb.com/title/tt0112346/
6110    https://www.imdb.com/title/tt1351685/
1278    https://www.imdb.com/title/tt0113028/
3271    https://www.imdb.com/title/tt0295238/
3479    https://www.imdb.com/title/tt0167260/
667     https://www.imdb.com/title/tt0420261/
3308    https://www.imdb.com/title/tt0279781/
4651                                      NaN
6158    https://www.imdb.com/title/tt1984204/
6366    https://www.imdb.com/title/tt0780548/
1247    https://www.imdb.com/title/tt0112729/
1200    https://www.imdb.com/title/tt0112401/
1534    https://www.imdb.com/title/tt0115738/
748     https://www.imdb.com/title

In [153]:
p = r'(tt\d{7})'

In [157]:
id = cleaned_movies_df['imdb_link'].str.extract(p)

In [178]:
try:
    cleaned_movies_df['imdb_id'] = cleaned_movies_df['imdb_link'].str.extract(p)
except Exception as e:
    print(e)

In [179]:
sorted(list(cleaned_movies_df.columns))

['Actor control',
 'Alias',
 'Alma mater',
 'Animation by',
 'Area',
 'Area served',
 'Artist(s)',
 'Attraction type',
 'Audio format',
 'Author',
 'Based on',
 'Biographical data',
 'Bopomofo',
 'Born',
 'Box office',
 'Budget',
 'Characters',
 'Children',
 'Cinematography',
 'Closing date',
 'Color process',
 'Comics',
 'Composer(s)',
 'Coordinates',
 'Country',
 'Cover artist',
 'Created by',
 'Date premiered',
 'Designer(s)',
 'Developer(s)',
 'Dewey Decimal',
 'Died',
 'Director',
 'Distributor',
 'Divisions',
 'Duration',
 'Editor(s)',
 'Engine',
 'Engine(s)',
 'Executive producer(s)',
 'Family',
 'Fate',
 'Film(s)',
 'Followed by',
 'Format(s)',
 'Formerly',
 'Founded',
 'Founder',
 'Founders',
 'Full name',
 'Gender',
 'Genre',
 'Genre(s)',
 'Genres',
 'Gwoyeu Romatzyh',
 'Hanyu Pinyin',
 'Headquarters',
 'Height',
 'Hokkien POJ',
 'IPA',
 'ISBN',
 'Illustrator',
 'Industry',
 'Instruments',
 'Jyutping',
 'Key people',
 'LC Class',
 'Label',
 'Labels',
 'Language',
 'Literal me

In [180]:
len(cleaned_movies_df)

7305

In [181]:
cleaned_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
len(cleaned_movies_df)

7050

In [241]:
columns_to_keep = [i for i in cleaned_movies_df.columns if cleaned_movies_df[i].isna().sum() != len(cleaned_movies_df)]


In [242]:
wiki_movies_df = cleaned_movies_df.loc[:, columns_to_keep]

In [248]:
print(len(list(cleaned_movies_df.columns)))
print(len(list(wiki_movies_df.columns)))

152
80


In [261]:
wiki_movies_df[wiki_movies_df['Box office'].isna()]

,url,year,imdb_link,title,Based on,Starring,Narrated by,Cinematography,Release date,Running time,...,Alias,Species,Gender,Family,Alma mater,Film(s),Original work,Television series,imdb_id,imdb_id2
5,https://en.wikipedia.org/wiki/The_Ambulance,1990.0,https://www.imdb.com/title/tt0099026/,The Ambulance,NaN,"[Eric Roberts, James Earl Jones, Red Buttons, ...",NaN,Jacques Haitkin,"[March 22, 1990, (, 1990-03-22, )]",95 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099026,tt0099026
6,https://en.wikipedia.org/wiki/American_Dream_(...,1990.0,https://www.imdb.com/title/tt0099028/,American Dream,NaN,NaN,NaN,"[Tom Hurwitz, Mathieu Roberts, Nesya Shapiro]","[October 6, 1990, (, 1990-10-06, ), (, New Yor...",100 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099028,tt0099028
7,https://en.wikipedia.org/wiki/American_Ninja_4...,1990.0,https://www.imdb.com/title/tt0101326/,American Ninja 4: The Annihilation,NaN,"[Michael Dudikoff, David Bradley, James Booth,...",NaN,NaN,"[March 8, 1991, (, 1991-03-08, )]",99 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0101326,tt0101326
8,https://en.wikipedia.org/wiki/Andre%27s_Mother,1990.0,https://www.imdb.com/title/tt0099037/,Andre's Mother,NaN,"[Richard Thomas, Sada Thompson, Sylvia Sidney]",NaN,Bobby Bukowski,"[March 7, 1990, (, 1990-03-07, )]",50 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099037,tt0099037
16,https://en.wikipedia.org/wiki/Basket_Case_2,1990.0,https://www.imdb.com/title/tt0099108/,Basket Case 2,NaN,Kevin Van Hentenryck,NaN,Robert M. Baldwin,"[March 2, 1990, (, 1990-03-02, )]",90 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099108,tt0099108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7157,https://en.wikipedia.org/wiki/Social_Animals_(...,2018.0,https://www.imdb.com/title/tt5639898/,Social Animals,NaN,"[Noël Wells, Josh Radnor, Aya Cash, Carly Chai...",NaN,Sandra Valde-Hansen,"[June 1, 2018, (, 2018-06-01, )]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt5639898,tt5639898
7170,https://en.wikipedia.org/wiki/The_Domestics,2018.0,https://www.imdb.com/title/tt5591666/,The Domestics,NaN,"[Tyler Hoechlin, Kate Bosworth, Lance Reddick,...",NaN,Maxime Alexandre,"[June 28, 2018, (, 2018-06-28, ), (United Stat...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt5591666,tt5591666
7261,https://en.wikipedia.org/wiki/Bullitt_County_(...,2018.0,https://www.imdb.com/title/tt6140148/,Bullitt County,NaN,"[Mike C. Nelson, Jenni Melear, David McCracken...",NaN,Sean McDaniel,"[October 26, 2018, (, 2018-10-26, ), (United S...",98 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt6140148,tt6140148
7284,https://en.wikipedia.org/wiki/Mowgli:_Legend_o...,2018.0,https://www.imdb.com/title/tt2388771/,Mowgli: Legend of the Jungle,"[All the Mowgli Stories, by, Rudyard Kipling]","[Christian Bale, Cate Blanchett, Benedict Cumb...",NaN,Michael Seresin,"[November 25, 2018, (, 2018-11-25, ), (, Mumba...",104 minutes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt2388771,tt2388771


In [280]:
a = wiki_movies_df['Box office']
len(a)

7050

In [286]:
box_office = wiki_movies_df['Box office'].dropna()
len(box_office)

5484

In [287]:
box_office[box_office.map(lambda x: type(x) != str)]

34                           [US$, 4,212,828]
56      [$6,698,361 (, United States, ), [2]]
76                    [$6,488,144, (US), [1]]
128                [US$1,531,489, (domestic)]
132                          [US$, 4,803,039]
                        ...                  
7206               [$99.6, million, [4], [5]]
7220                   [$365.6, million, [1]]
7221                         [$53.8, million]
7243                     [$435, million, [7]]
7277                   [$529.3, million, [4]]
Name: Box office, Length: 135, dtype: object

In [288]:
box_office[1:50]

1                              $2.7 million
2                               $57,718,089
3                                $7,331,647
4                          $6,939,946 (USA)
9                                  $855,810
10                           $195.3 million
11                            $53.2 million
12                            $15.7 million
13                            $52.1 million
14                           $244.5 million
15                              $12,626,043
17                              $19,740,070
18                             $138,697,012
20                               $1,292,323
21                             $8.2 million
22                            $15.6 million
24                               $2,070,871
25            $27.6 million (United States)
27                                  $10,173
29                               $5 million
33                            $35.8 million
34                         [US$, 4,212,828]
37                             $

In [283]:
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)
len(box_office)

5484

In [284]:
box_office[1:50]

1                         $2.7 million
2                          $57,718,089
3                           $7,331,647
4                     $6,939,946 (USA)
9                             $855,810
10                      $195.3 million
11                       $53.2 million
12                       $15.7 million
13                       $52.1 million
14                      $244.5 million
15                         $12,626,043
17                         $19,740,070
18                        $138,697,012
20                          $1,292,323
21                        $8.2 million
22                       $15.6 million
24                          $2,070,871
25       $27.6 million (United States)
27                             $10,173
29                          $5 million
33                       $35.8 million
34                       US$ 4,212,828
37                        $2.4 million
39                            $947,306
40                            $715,983
42                       

In [289]:
form_one = r'\$\d+\.?\d*\s*[mb]illion'
form_two = r'\$\d{1,3}(?:,\d{3})+'

In [290]:
def parse_dollars(s):
    # if s is not a string, return NaN
    if type(s) != str:
        return np.nan

    # if input is of the form $###.# million
    if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " million"
        s = re.sub('\$|\s|[a-zA-Z]', '', s)

        # convert to float and multiply by a million
        value = float(s) * 10**6

        # return value
        return value

    # if input is of the form $###.# billion
    elif re.match(r'\$\s*\d.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"
        s = re.sub('\$|\s|[a-zA-Z]', '', s)

        # convert to float and multiply by a billion
        value = float(s) * 10**9

        # return value
        return value

    # if input is of the form $###,###,###
    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

        # remove dollar sign and commas
        s = re.sub('\$|,', '', s)

        # convert to float
        value = float(s)

        # return value
        return value

    # otherwise, return NaN
    else:
        return np.nan

In [298]:
wiki_movies_df['Box office'].head(50)

0                     $21.4 million
1                      $2.7 million
2                       $57,718,089
3                        $7,331,647
4                  $6,939,946 (USA)
5                               NaN
6                               NaN
7                               NaN
8                               NaN
9                          $855,810
10                   $195.3 million
11                    $53.2 million
12                    $15.7 million
13                    $52.1 million
14                   $244.5 million
15                      $12,626,043
16                              NaN
17                      $19,740,070
18                     $138,697,012
19                              NaN
20                       $1,292,323
21                     $8.2 million
22                    $15.6 million
23                              NaN
24                       $2,070,871
25    $27.6 million (United States)
26                              NaN
27                          

In [299]:
wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)



In [301]:
wiki_movies_df[['Box office', 'box_office']].sample(25)

,Box office,box_office
2418,$16.3 million,16300000.0
1936,$171.1 million,171100000.0
5652,$232.3 million,232300000.0
5020,$139 million,139000000.0
763,$22.9 million,22900000.0
7136,NaN,NaN
490,$215.9 million,215900000.0
6202,"$13,769",13769.0
4465,$459.4 million,459400000.0
2735,$15.9 million,15900000.0


In [309]:
wiki_movies_df['Budget'][1:50]

1                              $6 million
2                             $35 million
3                             $12 million
4                             $25 million
5                                     NaN
6                                     NaN
7                                     NaN
8                                     NaN
9                                     NaN
10                            $50 million
11                            $22 million
12                                    NaN
13                            $29 million
14                            $40 million
15                             $7 million
16    [$2,500,000, [, citation needed, ]]
17                                    NaN
18                            $20 million
19                                    NaN
20                                    NaN
21                                    NaN
22                            $47 million
23                                    NaN
24                                

In [305]:
budget1 = wiki_movies_df['Budget'].dropna()
budget2 = wiki_movies_df['Budget'].dropna()

In [ ]:
budget1 = budget1.apply(lambda x: ' '.join(x) if type(x) == list else x)

In [308]:
budget1[1:50]

1                          $6 million
2                         $35 million
3                         $12 million
4                         $25 million
10                        $50 million
11                        $22 million
13                        $29 million
14                        $40 million
15                         $7 million
16     $2,500,000 [ citation needed ]
18                        $20 million
22                        $47 million
25                        $15 million
27                        $10 million
29                        $10 million
33                        $13 million
37                       $5.2 million
39                      $17.5 million
42                      US$18 million
43                        $12 million
44                         $4 million
45                        $22 million
48                        $16 million
50                        $60 million
51                       $6.5 million
56                        $16 million
58          

In [ ]:
budget2 = budget2.map(lambda x: ' '.join(x) if type(x) == list else x)

In [310]:
budget2[0:50]

0                         $20 million
1                          $6 million
2                         $35 million
3                         $12 million
4                         $25 million
10                        $50 million
11                        $22 million
13                        $29 million
14                        $40 million
15                         $7 million
16     $2,500,000 [ citation needed ]
18                        $20 million
22                        $47 million
25                        $15 million
27                        $10 million
29                        $10 million
33                        $13 million
37                       $5.2 million
39                      $17.5 million
42                      US$18 million
43                        $12 million
44                         $4 million
45                        $22 million
48                        $16 million
50                        $60 million
51                       $6.5 million
56          

In [312]:
budget = wiki_movies_df['Budget'].dropna()
budget = budget.apply(lambda x: ' '.join(x) if type(x) == list else x)
budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
budget = budget.str.replace(r'\[\d+\]\s*', '', regex=True)
wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

In [314]:
wiki_movies_df[['Budget', 'budget']]

,Budget,budget
0,$20 million,20000000.0
1,$6 million,6000000.0
2,$35 million,35000000.0
3,$12 million,12000000.0
4,$25 million,25000000.0
...,...,...
7300,$42 million,42000000.0
7301,$60 million,60000000.0
7302,$20 million,20000000.0
7303,$9 million,9000000.0
